In [1]:
import numpy as np
import pandas as pd
import copy

In [54]:
train_df = pd.read_excel("训练.xlsx")
test1_df = pd.read_excel("测试A.xlsx")
test2_df = pd.read_excel("测试B.xlsx")

In [55]:
new_column = test1_df.columns

In [56]:
train_y = train_df[train_df.columns[-1]]
train_y.shape

(500,)

In [57]:
train_df = train_df[new_column]

In [58]:
train_id = train_df['ID']
test1_id = test1_df['ID']
test2_id = test2_df['ID']
train_df.drop(['ID'], axis=1, inplace=True)
test1_df.drop(['ID'], axis=1, inplace=True)
test2_df.drop(['ID'], axis=1, inplace=True)

In [59]:
names = train_df.columns
dts = train_df.dtypes

In [60]:
drop_names = []
ratio = 0.125
for i in range(len(names)):
    n = names[i]
    tp = dts[i]
    trd = train_df[n]
    ted = test1_df[n]
    if 'object' == str(tp):
        continue
    else:
        misstr = np.sum(trd.isnull())/500
        misste = np.sum(ted.isnull())/100
        if np.abs(misstr-misste) > ratio:
            drop_names.append(n)
        else:
            if 'int' in str(tp):
                continue
            trrd = trd[trd.notnull()]
            terd = ted[ted.notnull()]
            m1 = np.mean(trrd)
            s1 = np.std(trrd)
            m2 = np.mean(terd)
            s2 = np.std(terd)
            if np.abs(m1-m2)>ratio*np.abs(m1) and np.abs(s1-s2)>ratio*np.abs(s1):
                drop_names.append(n)
print(len(drop_names))

815


In [61]:
train_df.drop(drop_names, axis=1, inplace=True)
test1_df.drop(drop_names, axis=1, inplace=True)
test2_df.drop(drop_names, axis=1, inplace=True)

In [62]:
names = train_df.columns
dfs = train_df.dtypes

In [63]:
# ints or strings
ints = []
for i in range(len(names)):
    if 'int' in str(dfs[i]) or 'object' in str(dfs[i]):
        ints.append(names[i])
print(len(ints))

1749


In [64]:
df_X = pd.concat([train_df, test1_df, test2_df], axis=0)

In [65]:
remains1 = []
remains2 = []
tobedrop = []
for n in ints:
    lu1 = len(train_df[n].unique())
    lu2 = len(df_X[n].unique())
    if lu1 < 10 and lu1 > 1:
        remains1.append(n)
    if lu2 < 10 and lu2 > 1:
        remains2.append(n)
    if lu1 != lu2:
        tobedrop.append(n)
print(len(remains1), len(remains2), len(tobedrop))

405 473 766


In [66]:
print(len(set(tobedrop) & set(remains1)), len(set(tobedrop) & set(remains2)), len(set(tobedrop) & set(remains2) & set(remains1)))

111 179 97


In [67]:
toberemain = list(set(remains1) & set(remains2) - set(tobedrop))
print(len(toberemain))

294


In [68]:
def dict2list(dic:dict):
    ''' 将字典转化为列表 '''
    keys = dic.keys()
    vals = dic.values()
    lst = [(key, val) for key, val in zip(keys, vals)]
    return lst

def LastRemainJudge(column):
    unique = set(column)
    uf = {}
    for u in unique:
        uf[u] = 0
    for c in column:
        uf[c] += 1
    sl = sorted(dict2list(uf), key = lambda x:x[1], reverse=True)
    x = sl[0][1]/len(column)
    sl = np.array(sl)
    if x < 0.8:
        #print(sl[:, 1]/len(column))
        return True
    else:
        return False

In [69]:
lm1 = []
lm2 = []
for n in toberemain:
    if LastRemainJudge(train_df[n]):
        lm1.append(n)
    if LastRemainJudge(df_X[n]):
        lm2.append(n)
print(len(lm1), len(lm2), len(set(lm1)&set(lm2)))

185 189 183


In [70]:
names = df_X.columns
dts = df_X.dtypes

In [71]:
dfx_float_n = []
dfx_onehot_n = list(set(lm1) & set(lm2))
print(len(dfx_onehot_n))
for i in range(len(names)):
    n = names[i]
    d = dts[i]
    if 'float' in str(d):
        dfx_float_n.append(n)
print(len(dfx_float_n))
dfx_float = df_X[dfx_float_n]
dfx_onehot = df_X[dfx_onehot_n]
print(dfx_float.shape, dfx_onehot.shape)

183
5668
(721, 5668) (721, 183)


In [72]:
drop_names = []
for n in dfx_float_n:
    x = np.sum(dfx_float[n].isnull())
    if x > 300:
        drop_names.append(n)
print(len(drop_names))

64


In [75]:
dfx_float.drop(drop_names, axis=1, inplace=True)

/home/magnusterra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [76]:
means = []
stds = []
drop_names = []
names = dfx_float.columns
for n in names:
    c = dfx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    means.append(tmpm)
    stds.append(tmps)
    if tmps == 0:
        drop_names.append(n)
print(len(set(means)), len(set(stds)))
print(len(drop_names))

2180 2180
0


In [77]:
# not move sames first, fill nan
added = set()
sames = []
for i in range(len(names)):
    tmpsl = []
    tmpsl.append(i)
    for j in range(i+1, len(names)):
        if means[i] == means[j] and stds[i] == stds[j]:
            if i not in added:
                added.add(i)
            if j not in added:
                added.add(j)
                tmpsl.append(j)
    if len(tmpsl) > 1:
        sames.append(tmpsl)
print(len(sames))

704


In [78]:
names = dfx_float.columns
drop_names = []
for i in sames:
    for j in i[1:]:
        drop_names.append(names[j])
print(len(drop_names))
print(dfx_float.shape)

3157
(721, 5338)


In [79]:
import copy
trainx_float = copy.copy(dfx_float)
trainx_float.drop(drop_names, axis=1, inplace=True)
print(trainx_float.shape)

(721, 2181)


In [80]:
names = trainx_float.columns
print(trainx_float.shape)
drop_names = []
for n in names:
    c = trainx_float[n]
    r = c[c.notnull()]
    tmpm = np.mean(r)
    tmps = np.std(r)
    if tmps>0:
        trainx_float[n][c.isnull()] = tmpm
    else:
        drop_names.append(n)
print(np.sum(trainx_float.isnull().values), len(drop_names))

(721, 2181)
0 0


In [81]:
trainx_float.drop(drop_names, axis=1, inplace=True)
print(np.sum(trainx_float.isnull().values))

0


In [82]:
trainx_float.shape

(721, 2181)

In [83]:
# it shall be that there is only thre lines with nan in log1p
mid = trainx_float-np.min(trainx_float)+1
trainx_log1p = np.log1p(mid)
del mid
print(np.sum(trainx_log1p.isnull().values))

0


In [84]:
names = trainx_float.columns
for n in names:
    m1 = np.mean(trainx_float[n])
    s1 = np.std(trainx_float[n])
    assert s1 > 0
    trainx_float[n] = (trainx_float[n]-m1)/s1
    m2 = np.mean(trainx_log1p[n])
    s2 = np.std(trainx_log1p[n])
    assert s2 > 0
    trainx_log1p[n] = (trainx_log1p[n]-m2)/s2

In [85]:
# make onehots onehot
ohnames = dfx_onehot.columns
for i in range(len(ohnames)):
    n = ohnames[i]
    tmp = pd.get_dummies(dfx_onehot[n], prefix='onehot_'+str(i))
    dfx_onehot = pd.concat([dfx_onehot, tmp], axis=1)
    print(dfx_onehot.shape)

(721, 192)
(721, 200)
(721, 204)
(721, 208)
(721, 215)
(721, 218)
(721, 223)
(721, 225)
(721, 231)
(721, 233)
(721, 235)
(721, 238)
(721, 242)
(721, 245)
(721, 247)
(721, 253)
(721, 256)
(721, 263)
(721, 268)
(721, 273)
(721, 279)
(721, 281)
(721, 283)
(721, 287)
(721, 291)
(721, 300)
(721, 304)
(721, 306)
(721, 309)
(721, 311)
(721, 314)
(721, 323)
(721, 326)
(721, 328)
(721, 331)
(721, 335)
(721, 337)
(721, 340)
(721, 349)
(721, 354)
(721, 358)
(721, 362)
(721, 370)
(721, 373)
(721, 376)
(721, 385)
(721, 387)
(721, 389)
(721, 392)
(721, 396)
(721, 399)
(721, 406)
(721, 408)
(721, 417)
(721, 419)
(721, 425)
(721, 431)
(721, 433)
(721, 439)
(721, 446)
(721, 450)
(721, 452)
(721, 457)
(721, 464)
(721, 469)
(721, 475)
(721, 478)
(721, 484)
(721, 487)
(721, 493)
(721, 497)
(721, 506)
(721, 513)
(721, 518)
(721, 523)
(721, 525)
(721, 529)
(721, 534)
(721, 536)
(721, 545)
(721, 548)
(721, 552)
(721, 554)
(721, 557)
(721, 559)
(721, 561)
(721, 564)
(721, 566)
(721, 571)
(721, 577)
(721, 584)

In [86]:
dfx_onehot.drop(ohnames, axis=1, inplace=True)
print(dfx_onehot.shape)

(721, 791)


In [87]:
dfx_float = trainx_float
dfx_log1p = trainx_log1p

In [42]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [88]:
names = dfx_float.columns
lr = LinearRegression()
lr.fit(dfx_float[:500], train_y)
fn = names[lr.coef_>0]
lr = LinearRegression()
lr.fit(dfx_log1p[:500], train_y)
ln = names[lr.coef_>0]

In [89]:
xfloatlr = dfx_float[fn]
xlog1plr = dfx_log1p[ln]

In [90]:
lr = LinearRegression()
lr.fit(dfx_onehot[:500], train_y)
print(np.sum(lr.coef_>0))
names = dfx_onehot.columns
xonehot = dfx_onehot[names[lr.coef_>0]]
print(xonehot.shape)

338
(721, 338)


In [46]:
def MSE(y_raw, y_pred):
    print(np.mean(np.square(y_raw-y_pred)))

In [47]:
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV 

/home/magnusterra/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [42]:
subxtrain, subxval, subytrain, subyval = train_test_split(dfx_log1p[:500].values, train_y.values, test_size=0.2, random_state=20)


In [43]:
tuned_parameters = {'objective':['reg:linear', 'reg:gamma'], 'max_depth':[1,2,3,4,5,6],
                    'gamma':[0, 1e-3, 1e-2,  1e-4], 'subsample':[0.85, 0.7]}
scores = ['MSE']
clf = GridSearchCV(estimator =XGBRegressor(), param_grid =tuned_parameters, cv=5, scoring='neg_mean_squared_error')
clf.fit(subxtrain, subytrain)
print("Best parameters set found on development set:")  
print()  
print(clf.best_params_)  
print()  
print("Grid scores on development set:")  
print()  
for params, mean_score, scores in clf.grid_scores_:  
    print("%0.3f (+/-%0.03f) for %r"  
              % (mean_score, scores.std() * 2, params))  
print()
print("The scores are computed on the full evaluation set.")  
print()  
y_true, y_pred = subyval, clf.predict(subxval)
print(MSE(y_true, y_pred))

Best parameters set found on development set:

{'gamma': 0.01, 'max_depth': 6, 'objective': 'reg:gamma', 'subsample': 0.85}

Grid scores on development set:

-0.039 (+/-0.005) for {'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'subsample': 0.85}
-0.038 (+/-0.007) for {'gamma': 0, 'max_depth': 1, 'objective': 'reg:linear', 'subsample': 0.7}
-0.038 (+/-0.005) for {'gamma': 0, 'max_depth': 1, 'objective': 'reg:gamma', 'subsample': 0.85}
-0.038 (+/-0.008) for {'gamma': 0, 'max_depth': 1, 'objective': 'reg:gamma', 'subsample': 0.7}
-0.038 (+/-0.005) for {'gamma': 0, 'max_depth': 2, 'objective': 'reg:linear', 'subsample': 0.85}
-0.038 (+/-0.005) for {'gamma': 0, 'max_depth': 2, 'objective': 'reg:linear', 'subsample': 0.7}
-0.037 (+/-0.005) for {'gamma': 0, 'max_depth': 2, 'objective': 'reg:gamma', 'subsample': 0.85}
-0.037 (+/-0.005) for {'gamma': 0, 'max_depth': 2, 'objective': 'reg:gamma', 'subsample': 0.7}
-0.037 (+/-0.007) for {'gamma': 0, 'max_depth': 3, 'objective': 'reg:linea

In [48]:
#xgb = XGBRegressor(gamma=0.01, max_depth=6, objective='reg:gamma', subsample=0.85)
nx = pd.concat([dfx_log1p, dfx_onehot], axis=1)
subxtrain, subxval, subytrain, subyval = train_test_split(nx[:500].values, train_y.values, test_size=0.2, random_state=0)



NameError: name 'get_oof' is not defined

In [49]:
xgb.fit(dfx_log1p[:500], train_y)
#test1_pred = Model_TripleTest(xfloatlr[:500], xlog1plr[:500], xexplr[:500], xonehot[:500], train_y, xfloatlr[500:600], xlog1plr[500:600], xexplr[500:600], xonehot[500:600])
test1_pred = xgb.predict(dfx_log1p[500:600])
save = pd.DataFrame({'ID':test1_id, 'value':test1_pred})
save.to_csv('answer_Synchronous_param_log_ratio15_xgb_20180109.csv', index=False, header=False)

In [49]:
from sklearn.model_selection import KFold 

In [50]:
kf = KFold(n_splits=5, random_state=2018)
def get_oof(rgr, xtrain, ytrain, xtest, lenot):
    oof_train = np.zeros((lenot,))
    oof_test = np.zeros((100,))
    oof_test_skf = np.empty((5, 100))
    for i, (train_index, test_index) in enumerate(kf.split(xtrain)):
        kf_xtrain = xtrain[train_index]
        kf_ytrain = ytrain[train_index]
        kf_xtest = xtrain[test_index]
        rgr.fit(kf_xtrain, kf_ytrain)
        oof_train[test_index] = rgr.predict(kf_xtest)
        oof_test_skf[i, : ] = rgr.predict(xtest)
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1,1), oof_test.reshape(-1,1)

In [91]:
#xgb = XGBRegressor(gamma=0.01, max_depth=6, objective='reg:gamma', subsample=0.85)
xgb = XGBRegressor()
tr_pred, te_pred = get_oof(xgb, subxtrain, subytrain, subxval, 400)
print(MSE(tr_pred, subytrain), MSE(te_pred, subyval))
xgb = XGBRegressor()
#tr_pred, te_pred = get_oof(xgb, subxtrain, subytrain, subxval)
tr_pred, te_pred = get_oof(xgb, nx[:500].values, train_y.values, nx[500:600].values, 500)
print(MSE(tr_pred, subytrain))

0.0515541856481
0.049627494265
None None
0.0505811417058
None


In [92]:
#tr_pred, te_pred = get_oof(xgb, subxtrain, subytrain, dfx_log1p[500:600].values, 500)
test1_pred = te_pred.reshape(100)
save = pd.DataFrame({'ID':test1_id, 'value':test1_pred})
save.to_csv('answer_Synchronous_paramstack_logratiooh125_xgb_20180109.csv', index=False, header=False)